In [1]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast
import acquire
import json
from igdb.wrapper import IGDBWrapper
import numpy as np

### Connect to API and scrape the data needed
#### get the access token and 200 response 

In [ ]:
def get_api():   
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *;'
    r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [2]:
access_token = acquire.connect_api()

In [3]:
tf = acquire.connect_to_games()

In [4]:
wrapper = acquire.run_wrapper()

In [ ]:
game_library = acquire.get_game_library(wrapper)
game_library.head()

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## scraping endpoints
scrape the end points of each category and add them to the game_librarys

### platform_families

In [ ]:
def get_platform_families():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_families' ,data=data ,  headers=headers)
    platform_families = wrapper.api_request('platform_families', 'fields *;limit 500;')
    x = json.loads(platform_families)
    platform_families =pd.DataFrame(x)
    return platform_families

In [ ]:
platform_families = get_platform_families()
platform_families.head()

In [ ]:
df.columns

### platform_version_release_dates

In [ ]:
def get_platform_version_release():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_release_dates' ,data=data ,  headers=headers)
    platform_version_release_dates = wrapper.api_request('platform_version_release_dates', 'fields *;limit 500;')
    x = json.loads(platform_version_release_dates)
    platform_version_release_dates =pd.DataFrame(x)
    return platform_version_release_dates

In [ ]:
platform_version_release_dates = get_platform_version_release()
platform_version_release_dates

In [ ]:
# drop columns not needed
platform_version_release_dates = platform_version_release_dates.drop(columns=['category', 'date', 'm', 'y', 'region', 'updated_at', 'checksum', 'updated_at'])
platform_version_release_dates.head()


In [ ]:
df.head()

In [ ]:
version_dates_dict[9] = 'not_available'
version_dates_dict[384] = 'not_available'
version_dates_dict[386] = 'not_available'


In [ ]:
version_dates_list = platform_version_release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
version_dates_dict = platform_version_release_dates.set_index('game').to_dict()['human']

In [ ]:
def test_funct3(random_list):
    if type(random_list) == list:
        return [version_dates_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
platforms.set_index('Locality', inplace=True)
data.set_index('Locality', inplace=True)


In [ ]:
df['release_dates'] = df.release_dates.apply(test_funct3)
df.head()

In [ ]:
all_release_dates = pd.merge(release_dates, platform_version_release_dates, how = 'inner', on="human")

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.head()

### platform_version_companies

In [ ]:
def get_platform_version_companies():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_companies' ,data=data ,  headers=headers)
    platform_version_companies = wrapper.api_request('platform_version_companies', 'fields *;limit 500;')
    x = json.loads(platform_version_companies)
    platform_version_companies =pd.DataFrame(x)
    return platform_version_companies

In [ ]:
platform_version_companies=get_platform_version_companies()
platform_version_companies.head()

### platforms (use this)

In [ ]:
def get_platforms():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platforms' ,data=data ,  headers=headers)
    platforms = wrapper.api_request('platforms', 'fields *;limit 500;')
    x = json.loads(platforms)
    platforms =pd.DataFrame(x)
    return platforms

In [ ]:
platform = get_platforms()
platform.shape

In [ ]:
# see if there is a number in id that matches the big df
platform.loc[platform['id'] == 5]

In [ ]:
platform = platform.drop(columns=['alternative_name', 'category', 'created_at', 'platform_logo', 'updated_at', 'url', 'checksum', 'versions', 'websites', 'generation', 'platform_family', 'abbreviation', 'summary'])
platform.head()


In [ ]:
# looking at the dictionary
print(game_library.get(92, "Not found"))

In [ ]:
platformlist = platform[['id' , 'name']].sort_values(by='id').reset_index(drop=True)


In [ ]:
platform_dict = platformlist.set_index('id').to_dict()['name']

In [ ]:
platform_dict[92] = 'not_available'

In [ ]:
def test_funct(random_list):
    if type(random_list) == list:
        return [platform_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
game_library['platforms'] = game_library.platforms.apply(test_funct)
game_library.head()

#### binning platforms

In [ ]:
game_library['platform_bin'] = pd.cut(game_library.platforms, 
                           bins = [0,10,20, 30, 40, 50, 60, 70, 80, 90, 100],
                           labels = ['awful','very_bad','bad','unimpressive','average','fair','alright','good','great', 'subperb'])



### release_dates

In [ ]:
def get_release_dates():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/release_dates' ,data=data ,  headers=headers)
    release_dates = wrapper.api_request('release_dates', 'fields *;limit 500;')
    x = json.loads(release_dates)
    release_dates =pd.DataFrame(x)
    return release_dates

In [ ]:
release_dates = get_release_dates()
release_dates.shape

In [ ]:
release_dates = release_dates.drop(columns=['category', 'date', 'm', 'region', 'updated_at', 'y', 'checksum', 'created_at', 'platform'])
release_dates.head()

In [ ]:
release_dates

In [ ]:
# add to dictionary
release_dict[91007] = "not-available"
release_dict[138333] = "not-available"
release_dict[188385] = "not-available"
release_dict[213594] = "not-available"
release_dict[77755] = "not-available"


In [ ]:
releaselist = release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
release_dict = releaselist.set_index('game').to_dict()['human']

In [ ]:
release_dates.head()

In [ ]:
def test_funct2(random_list):
    if type(random_list) == list:
        return [release_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
release_dict

In [ ]:
df.release_dates.head()

In [ ]:
df['release_dates'] = pd.DataFrame(df.release_dates.apply(test_funct2))
df

In [ ]:
release_dict

In [ ]:
# add 0 and description to dictionary
release_dict["0"] = "not-available"

### platform_versions

In [ ]:
def get_platform_versions():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_versions' ,data=data ,  headers=headers)
    platform_versions = wrapper.api_request('platform_versions', 'fields *;limit 500;')
    x = json.loads(platform_versions)
    platform_versions=pd.DataFrame(x)
    return platform_versions

In [ ]:
platform_versions = get_platform_versions()
platform_versions.shape

In [ ]:
platform_versions.head()

In [ ]:
df.columns

### player_perspectives (use this)

In [ ]:
def get_player_perspectives():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/player_perspectives' ,data=data ,  headers=headers)
    player_perspectives = wrapper.api_request('player_perspectives', 'fields *;limit 500;')
    x = json.loads(player_perspectives)
    player_perspectives=pd.DataFrame(x)
    return player_perspectives

In [ ]:
player_perspectives = get_player_perspectives()
player_perspectives.shape

In [ ]:
player_perspectives = player_perspectives.drop(columns=['created_at','updated_at', 'url', 'checksum'])
player_perspectives.head()


In [ ]:
player_perspectives_list = player_perspectives[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
player_perspectives_dict = player_perspectives.set_index('id').to_dict()['name']

In [ ]:
def test_funct4(random_list):
    if type(random_list) == list:
        return [player_perspectives_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
game_library['player_perspectives'] = pd.DataFrame(game_library.player_perspectives.apply(test_funct4))
game_library.head()

### themes (use this)

In [ ]:
def get_themes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/themes' ,data=data ,  headers=headers)
    themes = wrapper.api_request('themes', 'fields *;limit 500;')
    x = json.loads(themes)
    themes=pd.DataFrame(x)
    return themes

In [ ]:
themes = get_themes()
themes.shape

In [ ]:
themes_list = themes[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
themes_dict = themes.set_index('id').to_dict()['name']

In [ ]:
def test_funct5(random_list):
    if type(random_list) == list:
        return [themes_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
game_library['themes'] = pd.DataFrame(game_library.themes.apply(test_funct5))
game_library.head()

### Genres (use this)

In [ ]:
# function that puts response list object into a dataframe for each page
def get_genres(wrapper):
    genres = pd.DataFrame()
    for i in range (0, 409):
        genre = wrapper.api_request('genres', 'fields *; limit 500;')
        y = json.loads(genre)
        results_df =pd.DataFrame(y)
        genres = pd.concat([genres, results_df])
    return genres

In [ ]:
genres = get_genres(wrapper)
genres.head()

In [ ]:
genreslist = genres[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
genres_dict = genreslist.set_index('id').to_dict()['slug']

In [ ]:
def test_functg(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return  ["Not available"] 

In [ ]:
game_library['genres'] = pd.DataFrame(game_library.genres.apply(test_functg))
game_library.head()

## Game modes (use this)

In [ ]:
def get_game_modes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; limit 500;'
    r = requests.post('https://api.igdb.com/v4/game_modes' ,data=data ,  headers=headers)
    game_modes = wrapper.api_request('game_modes', 'fields *;limit 500;')
    x = json.loads(game_modes)
    game_modes =pd.DataFrame(x)
    return game_modes

In [ ]:
game_mode = get_game_modes()

In [ ]:
modelist = game_mode[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
mode_dict = modelist.set_index('id').to_dict()['slug']

In [ ]:
def test_functm(random_list):
    if type(random_list) == list:
        return [mode_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
game_library['game_modes'] = pd.DataFrame(game_library.game_modes.apply(test_functm))
game_library.head()

## Game version features (use this)

In [ ]:
def get_game_version_features(wrapper):
    game_version_features = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_features', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_features = pd.concat([game_version_features, results_df])
    return game_version_features

In [ ]:
game_version_features = get_game_version_features(wrapper)
game_version_features.head()

In [ ]:
game_vf_df = game_version_features[['id','description', 'title']]
game_vf_df.head()

In [ ]:
game_vf_df = game_vf_df.rename(columns={'description': 'version_description', 'title': 'title_version'})

## Game version feature values (use this)

In [ ]:
def vf_values(wrapper):
    vf_values= pd.DataFrame()
    for i in range (0, 409):
        gvf_values = wrapper.api_request('game_version_feature_values', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(gvf_values)
        results_df =pd.DataFrame(y)
        vf_values = pd.concat([vf_values, results_df])
    return vf_values

In [ ]:
gamevfv_df = vf_values(wrapper)
gamevfv_df.head()

In [ ]:
gamevfv_df = gamevfv_df[['game','game_feature']]
gamevfv_df.head()

In [ ]:
gamevfv_df = gamevfv_df.rename(columns={"game_feature": "id"})

In [ ]:
gamevfv_df = gamevfv_df.set_index('id')
gamevfv_df.head()

### Merging game feature version and game feature version values (use this)

In [ ]:
results_df = pd.merge(gamevfv_df,  game_vf_df , how = 'left', on="id")
results_df.head()

In [ ]:
results_df = results_df.rename(columns={'id': 'second','game': 'id' })
results_df = results_df.set_index('id')
results_df.head()

In [ ]:
#merging the to the main dataframe
game_library = pd.merge(game_library, results_df, how = 'left', on="id")
game_library.head()

In [ ]:
game_library.title_version_x.value_counts()

## Multi player mode (use this)

In [ ]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; limit 500;'
r = requests.post('https://api.igdb.com/v4/multiplayer_modes' ,data=data ,  headers=headers)
multiplayer_modes = wrapper.api_request('multiplayer_modes', 'fields *;limit 500;')
x = json.loads(multiplayer_modes)
multiplayer_modes =pd.DataFrame(x)

In [ ]:
def multi_play(wrapper):
    multi_play= pd.DataFrame()
    for i in range (0, 409):
        multi_values = wrapper.api_request('multiplayer_modes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(multi_values)
        results_df =pd.DataFrame(y)
        multi_play = pd.concat([multi_play, multiplayer_modes])
    return multi_play

In [ ]:
multi_df = multi_play(wrapper)

In [ ]:
def test_functg(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return  ["Not available"] 

In [ ]:
multi_df = multi_df.rename(columns={'game': 'id', 'id': 'second_id'})

In [ ]:
multi_df = multi_df.set_index('id')

In [ ]:
#marging this together with the main df
game_library = pd.merge(game_library, multi_df, how = 'left', on="id")
game_library.head()

In [ ]:
game_library['campaigncoop'].fillna(False, inplace = True)
game_library['dropin'].fillna(False, inplace = True)
game_library['lancoop'].fillna(False, inplace = True)
game_library['offlinecoop'].fillna(False, inplace = True)
game_library['offlinemax'].fillna(0, inplace = True)
game_library['onlinecoop'].fillna(False, inplace = True)
game_library['splitscreen'].fillna(False, inplace = True)
game_library['offlinecoopmax'].fillna(0, inplace = True)
game_library['onlinecoopmax'].fillna(0, inplace = True)
game_library['onlinemax'].fillna(0, inplace = True)

In [ ]:
game_library.head()

In [ ]:
game_library.shape

### loop through genres list

In [ ]:
genres_dict

In [ ]:
# list of genres to add
items = ['point-and-click', 'fighting', 'shooter', 'music', 'platform', 'puzzle', 'racing', 'real-time-strategy-rts', 'role-playing-rpg', 'simulator', 'sport', 'strategy', 'turn-based-strategy-tbs', 'tactical', 'hack-and-slash-beat-em-up', 'quiz-trivia', 'pinball', 'adventure', 'indie', 'arcade', 'visual-novel', 'card-and-board-game', 'moba']
# function to loop through column list and check for genre
def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [ ]:
for item in items:
    game_library[item] = game_library['genres'].apply(my_list, word=item)

In [ ]:
game_library.head()

In [ ]:
platform_dict

### Adding platforms to game_library

In [ ]:
len(platform_dict)

In [ ]:
# list of platformd to add
platforms = ['Linux', 'Nintendo 64', 'Wii', 'PC (Microsoft Windows)', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'Xbox', 'Xbox 360', 'DOS', 'Mac', 'Nintendo Entertainment System', 'Super Nintendo Entertainment System', 'Nintendo DS', 'Nintendo GameCube', 'Game Boy Color', 'Game Boy Advance', 'Sega Mega Drive/Genesis','Sega Saturn', 'Game Boy', 'Android', 'Nintendo 3DS','PlayStation Portable', 'iOS', 'Wii U', 'PlayStation Vita', 'Virtual Console (Nintendo)', 'PlayStation 4', 'Xbox One', 'Family Computer Disk System', 'Arcade', 'Windows Phone', 'Apple II', 'Web browser', 'Odyssey', 'Commodore 16', 'New Nintendo 3DS']
 

In [ ]:
# function to loop through column list and check for genre
def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [ ]:
for platform in platforms:
    game_library[platform] = game_library['platforms'].apply(my_list, word=platform)

In [ ]:
game_library.head()

### Adding themes to game library 

In [ ]:
themes_dict

In [ ]:
themes = ['Thriller', 'Science fiction', 'Action', 'Horror', 'Survival', 'Fantasy', 'Historical', 'Stealth', 'Comedy', 'Business', 'Drama', 'Non-fiction', 'Kids', 'Sandbox', 'Open world', 'Warfare', 'Educational', 'Mystery', 'Party', 'Romance', 'Erotic']

def my_list_themes(column, word):
    if word in column:
        return 1
    else:
        return 0


In [ ]:
for theme in themes:
    game_library[theme] = game_library['themes'].apply(my_list_themes, word=theme)

In [ ]:
game_library.head()

### Add player perspective 

In [ ]:
player_perspectives_dict

In [ ]:
players = ['First person', 'Third person', 'Bird view / Isometric', 'Text', 'Side view', 'Virtual Reality', 'Auditory'  'Drama']

def my_list_player(column, word):
    if word in column:
        return 1
    else:
        return 0


In [ ]:
for player in players:
    game_library[player] = game_library['player_perspectives'].apply(my_list_player, word=player)

In [ ]:
game_library.head()

In [ ]:
game_library.shape

### Add game modes

In [ ]:
mode_dict

In [ ]:
modes = ['single-player', 'multiplayer', 'co-operative', 'split-screen', 'massively-multiplayer-online-mmo', 'battle-royale']

def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [ ]:
for mode in modes:
    game_library[mode] = game_library['player_perspectives'].apply(my_list, word=mode)

In [ ]:
game_library.head()

In [ ]:
game_library.shape

### turn dlc column into boolean column

In [ ]:
game_library['dlcs'] = game_library['dlcs'].fillna(0)
game_library.head()

In [ ]:
def dlcs_col(df):
    game_library['has_dlcs'] = np.where(game_library.dlcs == 0, 1, 0)  
    return df

In [ ]:
game_library = dlcs_col(game_library)

In [ ]:
game_library.has_dlcs.head()

#### Delete columns we dont need

In [ ]:
game_library.head()

In [ ]:
game_library = game_library.drop(columns=['websites', 'external_games', 'updated_at', 'artworks', 'genres', 'platforms', 'similar_games', 'tags', 'release_dates', 'cover','screenshots', 'videos', 'checksum_x', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'url', 'franchises', 'follows', 'aggregated_rating_count', 'game_modes', 'player_perspectives', 'themes', 'game_engines', 'keywords', 'storyline', 'version_title', 'collection', 'follows', 'bundles', 'franchise', 'checksum_y'])
game_library.shape


In [ ]:
game_library.head()

In [ ]:
game_library = game_library.drop(columns=['websites', 'created_at', 'updated_at', 'artworks', 'cover','screenshots', 'videos', 'checksum_x', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'franchises', 'follows', 'aggregated_rating_count'])
game_library.shape

In [ ]:
game_library.head()

In [ ]:
game_library.rename(columns={'slug': 'lower_case_name', 'aggregated_rating': 'critic_rating'}, inplace=True)
game_library.head()

In [ ]:
game_library.title_version_x.value_counts()

### Dataframe with ratings

In [ ]:
df2 = game_library[game_library['rating'].notnull()]
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.rating.isnull().sum()

### Dataframe with nulls in rating column 

In [ ]:
df3 = game_library[game_library['rating'].isnull()]
df3.shape

## Explore

In [ ]:
game_library.head(20)

In [ ]:
game_library.player_perspectives.value_counts()